In [1]:
# ALWAYS RUN THIS CELL
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

In [2]:
import os
from pathlib import Path

import paramiko
from scp import SCPClient

import getpass
import gc
import time

In [3]:
dir_github = '/media/rich/Home_Linux_partition/github_repos/'
import sys
sys.path.append(dir_github)

%load_ext autoreload
%autoreload 2
from basic_neural_processing_modules import server

----
# ==== Optional ====
### transfer raw data from local machine to `MICROSCOPE`

In [4]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = server.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


In [13]:
sftp = server.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=server.pw_decode(pw))

In [7]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    passcode_method=1,
    verbose=1,
    skip_passcode=False,    
)

Exception ignored in: <function ssh_interface.__del__ at 0x7ff7b4093310>
Traceback (most recent call last):
  File "/media/rich/Home_Linux_partition/github_repos/basic_neural_processing_modules/server.py", line 370, in __del__
    self.ssh.close()
AttributeError: 'ssh_interface' object has no attribute 'ssh'


Duo two-factor login for rh183

Enter a passcode or select one of the following options:

 1. Duo Push to XXX-XXX-4151
 2. Phone call to XXX-XXX-4151
 3. SMS passcodes to XXX-XXX-4151

Passcode or option (1-3): 
1



Pushed a login request to your device...

Success. Logging you in...
Welcome to O2 (Orchestra 2)!

   ___    ____
  / _ \  |___ \
 | | | |   __) |
 | |_| |  / __/
  \___/  |_____|


You've landed on transfer03 which is a
4 core system with 7.62 GiB memory
running kernel 3.10.0 born on 2020-07-31

==== O2 ===================================================================

  News (Jan 3 2022)
  

  +-------------------------------------------------------------------------------+
  | ##################### RC Core launched in July 2021 ######################### |
  |                                                                               |
  | Since July 2021, researchers whose PI does not have a primary or secondary    |
  | faculty appointment in an HMS Basic or Socia

In [16]:
## make directory structure on MICROSCOPE
animals = ['AEG21', 'AEG22']
dates = ['2022_05_13', '2022_05_14', '2022_05_16', '2022_05_17', '2022_05_18']
methods = ['clean_traces', 'ROI_classification_logisticRegression', 'spike_inference', 'suite2p']

[[[sftp.mkdir_p(str(Path('/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/') / animal / date / method)) for animal in animals] for date in dates] for method in methods]

[[[True, True], [True, True], [True, True], [True, True], [True, True]],
 [[True, True], [True, True], [True, True], [True, True], [True, True]],
 [[True, True], [True, True], [True, True], [True, True], [True, True]],
 [[True, True], [True, True], [True, True], [True, True], [True, True]]]

In [25]:
## make directory structure on MICROSCOPE
animals = ['AEG21', 'AEG22']
dates = ['2022_05_13', '2022_05_14', '2022_05_16', '2022_05_17', '2022_05_18']
methods_MICROSCOPE = ['clean_traces', 'ROI_classification_logisticRegression', 'spike_inference', 'suite2p_output']
methods_o2 = ['cleanTraces', 'classify_rois', 'spike_inference', 'suite2p_output']

[[[sftp.mkdir_p(str(Path('/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/') / animal / date / method)) for animal in animals] for date in dates] for method in methods_MICROSCOPE]

[[[True, True], [True, True], [True, True], [True, True], [True, True]],
 [[True, True], [True, True], [True, True], [True, True], [True, True]],
 [[True, True], [True, True], [True, True], [True, True], [True, True]],
 [[True, True], [True, True], [True, True], [True, True], [True, True]]]

In [26]:
## Define directories for where the local data is and the remote directory where the contents of that folder should be copied to (MICROSCOPE)
dir_data_o2 = '/n/data1/hms/neurobio/sabatini/rich/analysis/'
dir_data_MICROSCOPE = '/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM'  ## contents of dir_data_local will be dumped into dir_data_MICROSCOPE

[[[ssh_t.send_receive(f"cp -r {str(Path(dir_data_o2) / method_o2 / animal / date / 'jobNum_0/.')} {str(Path(dir_data_MICROSCOPE) / animal / date / method_MICROSCOPE)}", timeout=30, post_send_wait_t=10) for method_o2,method_MICROSCOPE in zip(methods_o2, methods_MICROSCOPE)] for animal in animals] for date in dates]

sis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_13/suite2p_outputRich/analy 
t
_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_13/clean_tracesE/Rich/analysis 
s
icRegression_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_13/ROI_classification_logist 
n
ysis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_13/spike_inferenceich/anal 
sis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_13/suite2p_outputRich/analy 
_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_14/clean_tracesE/Rich/analysis 
icRegression_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_14/ROI_classification_logist 
ysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_14/spike_inferenceich/anal 
sis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_14/suite2p_outputRich/analy 
_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_14/clean_tracesE/Rich/analysis 
icRegression_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_14/ROI_classification_logist 
ysis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_14/spike_inferenceich/anal 
sis_data/f

[[['(base) [rh183@transfer03 ~]$ cp -r /n/data1/hms/neurobio/sabatini/rich/analysis/ \rsuite2p_output/AEG21/2022_05_13/jobNum_0 /n/files/Neurobio/MICROSCOPE/Rich/analy \rsis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_13/suite2p_output\r\nt\r\n(base) [rh183@transfer03 ~]$ cp -r /n/data1/hms/neurobio/sabatini/rich/analysis/ \rcleanTraces/AEG22/2022_05_13/jobNum_0 /n/files/Neurobio/MICROSCOPE/Rich/analysis \r_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_13/clean_traces\r\ns\r\n(base) [rh183@transfer03 ~]$ cp -r /n/data1/hms/neurobio/sabatini/rich/analysis/ \rclassify_rois/AEG22/2022_05_13/jobNum_0 /n/files/Neurobio/MICROSCOPE/Rich/analys \ris_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_13/ROI_classification_logist \ricRegression\r\nn\r\n(base) [rh183@transfer03 ~]$ cp -r /n/data1/hms/neurobio/sabatini/rich/analysis/ \rspike_inference/AEG22/2022_05_13/jobNum_0 /n/files/Neurobio/MICROSCOPE/Rich/anal \rysis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_13/spike_inference\r\